# 맵리듀스

맵리듀스(MapReduce)는 빅데이터를 병렬 처리할 때 사용하는 프로그래밍 모델이다. 성능은 굉장하지만 원리 자체는 아주 간단하다.

여러 아이템으로 구성된, 처리하고 싶은 데이터셋이 있다고 해보자. 그 데이터셋은 웹사이트 로그일 수 도 있고, 수많은 책의 텍스트일 수도 있고, 이미지 파일 또는 다른 무엇이 될 수도 있다. 맵리듀스 알고리즘의 기본적인 원리는 다음과 같다.

1. mapper 함수로 각 아이템을 0개 이상의 키-값 쌍으로 변환한다. (때로는 이것을 단순히 map 함수라고 부르기도 하지만, 파이썬에는 이미 map이라는 함수가 있으니 두 가지를 혼동하지 않기 위해서 mapper라고 부르겠다.)

2. 동일한 키를 가진 모든 키-값 쌍을 모은다.

3. reducer 함수로 같은 키를 가진 쌍들에 대해 원하는 방식으로 처리된 값을 반환한다.

## 25.1 예시: 단어 수  세기
사용자들이 올리는 게시물에 등장하는 단어들의 수를 세서, 빈도 수가 가장 높은 단어들을 찾아 제시해 보자.

In [15]:
from typing import List
from collections import Counter

def tokenize(document: str) -> List[str]:
    """공백 문자를 기준으로 나눈다."""
    return document.split()

In [16]:
def word_count_old(document: List[str]):
    """맵리듀스를 사용하지 않고 단어 수 세기"""
    return Counter(word
                  for document in documents
                  for word in tokenize(document))

사용자의 수가 많아지면 문서(documents)의 양이 너무 많아서 컴퓨터 한 대로는 부족할 정도가 되었다. 데이터를 맵리듀스 모델 안에 넣을 수만 있다면, 엔지니어들이 구축해 놓은 '빅 데이터' 인프라를 사용할 수 있다.

먼저, 문서를 키-값 쌍으로 변환시켜 주는 함수가 필요하다. 최종 결과물은 단어별로 묵인 결과일 테니, 결국 키는 단어가 되어야 한다. 그리고 각 단어를 만날 때마다 한 번 등장했다는 것을 표시하기 위해, 값을 1로 설정하겠다.

In [17]:
from typing import Iterable, Tuple

def wc_mapper(document: str) -> Iterable[Tuple[str, int]]:
    """문서의 각 단어마다 (단어, 1)를 내보낸다."""
    for word in tokenize(document):
        yield (word, 1)

앞에서 언급한 2번째 단계는 잠시 건너뛰고, 특정 단어에 대한 등장 횟수를 리스트로 수집했다고 해보자. 그러면 단어 수의 총합은 다음과 같이 구할 수 있다.

In [18]:
from typing import Iterable

def wc_reducer(word:str,
              counts: Iterable[int]) -> Iterable[Tuple[str, int]]:
    """단어의 모든 빈도 수를 더한다."""
    yield (word, sum(counts))

이제 다시 2번째 단계로 돌아가 보면 wc_mapper로 부터 얻은 결과를 wc_reducer한테 전달해 주기만 하면 된다. 이것을 컴퓨터 한 대로 처리할 방법을 생각해보자.

In [19]:
from collections import defaultdict

def word_count(documents: List[str]) -> List[Tuple[str, int]]:
    """맵리듀스를 사용해서 입력 문서의 단어 빈도 수도를 세어 준다."""
    
    collector = defaultdict(list)  # 쌍으로 묶인 값을 저장할 공간
    
    for document in documents:
        for word, count in wc_mapper(document):
            collector[word].append(count)
            
    return [output
           for word, counts in collector.items()
           for output in wc_reducer(word, counts)]

문서가 ["data science", "big data", "science fiction"] 세 개 뿐이라고 해보자. 

첫 번째 문서에 적용된 wc_mapper는 ("data",1)과 ("science", 1)이라는 결과를 반환해 준다. 모든 문서에 대해 동일한 함수를 적용하면, collector는 다음과 같은 결과를 얻게 된다.

In [20]:
documents = ["data science", "big data", "science fiction"]
word_count(documents)

[('data', 2), ('science', 2), ('big', 1), ('fiction', 1)]

## 25.2 왜 맵리듀스인가?

맵리듀스의 가장 **큰 장점**은 데이터 처리(함수)를 데이터로 가져가서 한다는 것이다. 수십억 개의 문서에서 단어의 수를 세어 보자.

맵리듀스가 아닌 기존의 방법을 사용하면 컴퓨터 한 대로 모든 문서를 한 번씩 접근해야 한다. 즉, 모든 문서가 그 컴퓨터의 저장 공간 안에 있거나, 프로세싱 도중 그 컴퓨터에게 전달되어야 한다는 것을 의미한다. 여기서 중요한 것은 그 컴퓨터가 문서를 한 번에 하나밖에 처리하지 못한다는 것이다. 

이번에는 수십억 개의 문서가 100대의 컴퓨터에 흩어져 있다고 해보자. 제대로된 인프라만 갖춰져 있다면(그리고 세부적인 부분은 조금 무시한다면) 우리는 다음과 같은 문서를 처리할 수 있다. =

- 각 컴퓨터는 자신이 보유하고 있는 문서에 mapper를 수행하고, 여러 개의 키-값 쌍을 생성한다.

- 키-값 쌍들을 지정된 몇 개의 리듀스 작업을 수행하는 컴퓨터로 보낸다. 단, 같은 키를 가진 쌍들은 모두 동일한 컴퓨터에 모이게 한다.

- 리듀스 작업을 수행하는 각 컴퓨터는 키별로 상들을 묶고 reducer를 실행 한다.

- 각 키-최종 값 쌍을 반환한다.

이 방법이 우수한 이유는 수평적 확장이 용이하기 때문이다. 즉, 컴퓨터의 개수를 두 배로 늘린다면(맵리듀스 시스템상 몇 가지 고정비용만 제외한다면) 계산도 두 배나 빨라질 것이다. 각 mapper 컴퓨터는 절반의 일만 하면 되고 (reducer의 일을 더 분산시킬 수 있을 정도로 충분한 수의 키가 존재한다는 가정하에) reducer 컴퓨터 또한 마찬가지로 절반의 일만 하면 된다.

## 25.3 맵리듀스 일반화하기

일반적인 타입을 사용하여 위의 map_reduce 함수에 타입 어노테이션을 완전히 적용할 수 있을 것이다.

In [22]:
from typing import Callable, Iterable, Any, Tuple

# 키-값 쌍은 그저 값이 두 개인 튜플이다.
KV = Tuple[Any, Any]

# Mapper는 키-값 쌍의 이터러블을 반환하는 함수
Mapper = Callable[..., Iterable[KV]]

# Reducer는 키와 값의 이터러블을 받고
# 키-값 상을 반환하는 함수
Reducer = Callable[[Any, Iterable], KV]

일반화된 map-reduce 함수를 작성

In [23]:
def map_reduce(inputs: Iterable, 
              mapper: Mapper,
              reducer: Reducer) -> List[KV]:
    """mapper와 reducer를 사용해서 inputs에 맵리듀스를 적용"""
    collector = defaultdict(list)
    
    for input in inputs:
        for key, value in mapper(input):
            collector[key].append(value)
            
    return [output
           for key, values in collector.items()
           for output in reducer(key, values)]

위 코드를 이용하면 아래와 같이 단어를 셀 수 있다.

In [25]:
word_counts = map_reduce(documents, wc_mapper, wc_reducer)

In [26]:
print(word_counts)

[('data', 2), ('science', 2), ('big', 1), ('fiction', 1)]


wc_reducer는 각 키에 해당하는 값들을 단순히 더해 주는 일만 한다는 점을 유심히 살펴보자. 이런 방식의 병합은 아주 빈번하게 발생하는 작업이라 이 역시 일반화할 수 있다.

In [27]:
def values_reducer(values_fn: Callable) -> Reducer:
    """values_fn을 값에 적용하는 reducer 반환"""
    def reduce(key, values: Iterable) -> KV:
        return (key, values_fn(values))
    
    return reduce

In [29]:
sum_reducer = values_reducer(sum)
max_reducer = values_reducer(max)
min_reducer = values_reducer(min)
count_distinct_reducer = values_reducer(lambda values: len(set(values)))

assert sum_reducer("key", [1, 2, 3, 3]) == ("key", 9)
assert min_reducer("key", [1, 2, 3, 3]) == ("key", 1)
assert max_reducer("key", [1, 2, 3, 3]) == ("key", 3)
assert count_distinct_reducer("key", [1, 2, 3, 3]) == ("key", 3)

## 25.4 예시: 사용자의 글 분석하기

In [34]:
import datetime

status_updates = [
    {"id": 2,
     "username" : "joelgrus",
     "text" : "Should I write a second edition of my data science book?",
     "created_at" : datetime.datetime(2018, 2, 21, 11, 47, 0),
     "liked_by" : ["data_guy", "data_gal", "mike"] },
     # ...
]

In [35]:
def data_science_day_mapper(status_update: dict) -> Iterable:
    """status_update에 'data science'가 포함되어 있으면 (day_of_week, 1)을 내보낸다."""
    if "data science" in  status_update["text"].lower():
        day_of_week = status_update["created_at"].weekday()
        yield (day_of_week, 1)

In [36]:
data_science_days = map_reduce(status_updates,
                              data_science_day_mapper,
                               sum_reducer)

조금 더 복잡한 예시로 각 사용자가 어떤 단어를 가장 많이 사용했는지 분석할 수 있다. 이 경우, mapper를 다음과 같은 세 가지 방법으로 작성할 수 있다.

- 사용자의 이름을 키로 사용하고, 단어와 단어의 수를 값으로 사용한다.
- 단어를 키로 사용하고, 사용자의 이름과 단어의 수를 값으로 사용한다.
- 사용자의 이름과 단어를 키로 사용하고, 단어의 수를 값으로 사용한다.

조금 더 생각해 보면, 각 사용자의 단어들을 개별적으로 세기 위해서는 사용자의 이름을 키로 사용해야 한다. 그리고 단어를 키로 사용하는 것은 피해야 한다.

각 사용자별로 모든 단어의 빈도 수를 계산한 뒤, 가장 빈도가 높은 단어를 찾아야 하기 때문이다. 즉, 위의 세 가지 옵션 중에서 첫 번째 옵션이 가장 좋다.

In [37]:
def words_per_user_mapper(status_update: dict):
    user = status_update["username"]
    for word in tokenize(status_update["text"]):
        yield (user, (word, 1))

In [38]:
def most_popular_word_reducer(user: str,
                             words_and_counts: Iterable[KV]):
    """
    (단어, 빈도 수) 쌍으로 구성된 배열에서
    총 빈도 수가 가장 높은 단어를 반환
    """
    word_counts = Counter()
    for word, count in words_and_counts:
        word_counts[word] += count
        
    word, count = word_counts.most_common(1)[0]
    
    yield (user, (word, count))

In [39]:
user_words = map_reduce(status_updates,
                       words_per_user_mapper,
                       most_popular_word_reducer)

또한 각 사용자별로 글에 '좋아요'를 했던 다른 사용자 수를 셀 수도 있다.

In [40]:
def liker_mapper(status_update: dict):
    user = status_update["username"]
    for liker in status_update["liked_by"]:
        yield (user, liker)

In [41]:
distinct_liker_per_user = map_reduce(status_updates,
                                    liker_mapper,
                                    count_distinct_reducer)

## 25.5 예시 행렬 곱셈

22.2.1 절 '행렬 곱셈'에서 [n, m]행렬 A와 [m, k] 행렬 B가 주어졌을 때, 둘을 곱해 [n, k] 행렬 C를 구할 수 있는데, 이때 i번째 행과 j번째 열에 속하는 원소이 값은 다음과 같다.

C[i][j] = sum(A[i][j] * B[i][j] for x in range(m))

이는 우리가 지금까지 그래왔듯 행렬이 리스트와 리스트로 표현되면 가능하다.

하지만 가끔씩 큰 행렬의 경우, 대부분의 원소가 0인 희소(sparse)한 특성을 가질 때가 있다. 이렇게 큰 희소 행렬에 대해서는 리스트의 리스트 형태는 상당히 비효율적이다. **보다 효율적인 표현법은 0이 아닌 값을 담고 있는 위치만 저장하는 것이다.**

In [42]:
from typing import NamedTuple

class Entry(NamedTuple):
    name: str
    i: int
    j: int
    value: float

예를 들어 1억 x 1억 행렬은 1경 개의 원소를 가지며, 이렇게 많은 원소를 한 컴퓨터 안에 저장하기는 쉽지 않을 수 있다. 하지만 원소가 대대분의 0인 경우에는 이 대안적인 표현법이 저장 공간을 대폭 절약시켜줄 수 있을 것이다.

이러한 표현법이 있을 때, 맵리듀스를 이용해서 분산적으로 행렬의 곱을 구할 수 있다.

행각해 보면 각 원소 A[i][j]는 C의 i번째 행 그리고 B[i][j]는 C의 j번째 열에만 관여한다. reducer의 결괏값이 C의 각 원소가 되는 것이 우리의 목표라고 한다면, mapper 역시 C의 각 원소를 키로 출력하도록 해야 함을 의미한다. 

이는 아래와 같은 구현으로 이어진다.

In [50]:
def matrix_multiply_mapper(num_rows_a: int, num_cols_b: int) -> Mapper:
    # C[x][y] = A[x][0] * B[0][y] + ... + A[x][m] * B[m][y]
    #
    # 그러므로 A[i][j]는 모든 C[i][y]에 B[i][y]의 가중치로 들어간다.
    # 그리고 B[i][j]는 모든 C[x][j]에 A[x][i]의 가중치로 들어간다.
    def mapper(entry: Entry):
        if entry.name == "A":
            for y in range(num_cols_b):
                key = (entry.i, y)  # C의 어떤 원소
                value = (entry.j, entry.value)  # 합의 어떤 entry
                yield (key, value)
                
        else:
            for x in range(num_rows_a):
                key = (x, entry.j)  # C의 어떤 원소
                value = (entry.i, entry.value)  # 합의 어떤 entry
                yield (key, value)
                
    return mapper

그리고 나면 다음과 같다.

In [51]:
def matrix_multiply_reducer(key: Tuple[int, int],
                           indexed_values: Iterable[Tuple[int, int]]):
    results_by_index = defaultdict(list)
    
    for index, value in indexed_values:
        results_by_index[index].append(value)
        
        # 값이 두 개인 위치의 값을 곱하고
        #(A에서 하나, B에서 하나) 이를 더한다.
        sumproduct = sum(values[0] * values[1]
                        for values in results_by_index.values()
                        if len(values) == 2)
        
    if sumproduct != 0.0:
        yield (key, sumproduct)

예를 들어 아래와 같은 두 행렬이 있다면

In [52]:
A = [[3, 2, 0],
     [0, 0, 0]]

B = [[4, -1, 0],
     [10, 0, 0],
     [0, 0, 0]]


이를 튜플로 표현할 수 있다.

In [53]:
entries = [Entry("A", 0, 0, 3), Entry("A", 0, 1,  2), Entry("B", 0, 0, 4),
               Entry("B", 0, 1, -1), Entry("B", 1, 0, 10)]

mapper = matrix_multiply_mapper(num_rows_a=2, num_cols_b=3)
reducer = matrix_multiply_reducer

In [54]:
# 곱은 [[32, -3, 0], [0, 0, 0]] 이어야 한다.
# 그러므로 두 개의 항목이 있어야 한다.
assert (set(map_reduce(entries, mapper, reducer)) == {((0,1), -3), ((0,0), 32)})

이런 방법은 작은 행렬에서는 크게 효과적으이지 않지만, 수백만 개의 행과 수백만 개의 열이 주어졌을 때는 꽤 효과적이다.

## 25.6 여담: Combiner

mapper들이 꽤 많은 정보를 담고 있다는 것을 발견했을 것이다. 예를 들어 단어를 셀 때는 (word, 1)을 전달해서 값을 모두 더하기보다 차라리 (word, None)을 전달해서 단순히 리스트의 길이만 구할 수도 있다.

하지만 그렇게 하지 않은 이유가 잇다. 분산적인 환경에서는 컴퓨터와 컴퓨터 사이에서 전달되어야 하는 데이터의 양을 줄이기 위해 combiner를 사용할 수 있기 때문이다. 예를 들면 한 mapper 컴퓨터가 'data'라는 단어를 500번 보았을 때, 그 컴퓨터한테 500개의 ('data', 1)을 하나의 ("data", 500)으로 합친 후  리듀스 작업을 하는 컴퓨터에 전달하게 하는 것이 훨씬 효율적일 것이다. 이렇게 하는 것이 컴퓨터 간 데이터 전송량을 줄여 주기 때문에 전체적인 속도 또한 훨씬 빠르다.

Reducer를 앞에서와 같이 작성한 덕에 코드를 수정하지 않고 이렇게 합쳐진 데이터를 다룰 수 있다. 만약 리스트의 길이를 구하는 방법으로 접근햇다면 불가능했을 것이다.

## 25.7 더 공부해 보고 싶다면

- 하둡(Hadoop)은 가장 널리 사용되는 맵리듀스 시스템이다. 
- 아마존 AWS에서는 Elastic MapReduce(EMR)라는 서비스를 제공하는데, 직접 클러스터를 구성하는 것에 비해 훨씬 쉬울 것이다. 
- 하둡에는 보통 지연 속도가 있어서 실시간 분석에는 적합하지 않다. 그런 종류의 작업을 맵리듀스와 비슷하게 다루는 데는 스파크(Spark)가 인기있다.